# MongoDB database met landingen van meteorieten

De volgende regels zijn nodig voor sommige systemen:

In [1]:
import os
os.environ["PATH"] += ":/usr/local/bin"

We maken een aantal directories (data, data/db en log) waar we de output files van MongoDB opslaan. Daarna configureren we MongoDB en starten we de database.

In [2]:
%%bash
mkdir data
mkdir data/db
mkdir log

mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘data/db’: File exists
mkdir: cannot create directory ‘log’: File exists


CalledProcessError: Command 'b'mkdir data\nmkdir data/db\nmkdir log\n'' returned non-zero exit status 1.

In [15]:
%%bash
mongod --dbpath ./data/db --logpath ./log/mongodb.log
exec "$@"

2019-12-09T08:21:48.690+0000 I CONTROL  [main] log file "/home/jovyan/./log/mongodb.log" exists; moved to "/home/jovyan/./log/mongodb.log.2019-12-09T08-21-48".


### De JSON-file
We gebruiken een JSON-file met landingen van meteorieten op aarde. Deze JSON-file klopt niet volgens het originele JSON format, maar het bevat een hele lading kleine JSON-objects, gescheiden met een line feed. Met `cat` vragen we de contents van onze JSON-file op. Merk op dat er allemaal losse JSON-objects in staan (ze staan dus __niet__ in een array!).

In [3]:
%%bash
cat meteorite_landings_on_earth.json

{"name":"Aachen","id":"1","nametype":"Valid","recclass":"L5","mass":"21","fall":"Fell","year":"1880-01-01T00:00:00.000","reclat":"50.775000","reclong":"6.083330","geolocation":{"type":"Point","coordinates":[6.08333,50.775]}}
{"name":"Aarhus","id":"2","nametype":"Valid","recclass":"H6","mass":"720","fall":"Fell","year":"1951-01-01T00:00:00.000","reclat":"56.183330","reclong":"10.233330","geolocation":{"type":"Point","coordinates":[10.23333,56.18333]}}
{"name":"Abee","id":"6","nametype":"Valid","recclass":"EH4","mass":"107000","fall":"Fell","year":"1952-01-01T00:00:00.000","reclat":"54.216670","reclong":"-113.000000","geolocation":{"type":"Point","coordinates":[-113,54.21667]}}
{"name":"Acapulco","id":"10","nametype":"Valid","recclass":"Acapulcoite","mass":"1914","fall":"Fell","year":"1976-01-01T00:00:00.000","reclat":"16.883330","reclong":"-99.900000","geolocation":{"type":"Point","coordinates":[-99.9,16.88333]}}
{"name":"Achiras","id":"370","nametype":"Valid","recclass":"L6","mass":"78

### De database starten
We starten nu de database zodat we `find` op onze collection kunnen uitvoeren. De onderstaande bash-regel start onze MongoDB database zodat we de data kunnen opvragen met `db.landings.find`. Merk op dat we de collection `landings` noemen (`-c landings`): dit is gelijk aan het veld in `db` waar we `find` mee kunnen uitvoeren. Onze database heet `landingsdb`. In elke volgende cel voeren we `mongo landingsdb --quiet` uit om te verbindeng met onze database.

In [4]:
%%bash
mongoimport -d landingsdb --drop -c landings meteorite_landings_on_earth.json

2019-12-09T08:19:52.046+0000	connected to: localhost
2019-12-09T08:19:52.046+0000	dropping: landingsdb.landings
2019-12-09T08:19:52.086+0000	imported 1000 documents


### Operaties op onze database
Met `find` kunnen we de data opvragen aan de hand van een query. Deze query laten we tot nu toe even leeg: dit betekent dat we alle data opvragen, zonder enige verdere verwerkingen.

In [5]:
%%bash
mongo landingsdb --quiet

db.landings.find({})

{ "_id" : ObjectId("5dee03a8379ce4d5c6654798"), "name" : "Aachen", "id" : "1", "nametype" : "Valid", "recclass" : "L5", "mass" : "21", "fall" : "Fell", "year" : "1880-01-01T00:00:00.000", "reclat" : "50.775000", "reclong" : "6.083330", "geolocation" : { "type" : "Point", "coordinates" : [ 6.08333, 50.775 ] } }
{ "_id" : ObjectId("5dee03a8379ce4d5c6654799"), "name" : "Aarhus", "id" : "2", "nametype" : "Valid", "recclass" : "H6", "mass" : "720", "fall" : "Fell", "year" : "1951-01-01T00:00:00.000", "reclat" : "56.183330", "reclong" : "10.233330", "geolocation" : { "type" : "Point", "coordinates" : [ 10.23333, 56.18333 ] } }
{ "_id" : ObjectId("5dee03a8379ce4d5c665479a"), "name" : "Abee", "id" : "6", "nametype" : "Valid", "recclass" : "EH4", "mass" : "107000", "fall" : "Fell", "year" : "1952-01-01T00:00:00.000", "reclat" : "54.216670", "reclong" : "-113.000000", "geolocation" : { "type" : "Point", "coordinates" : [ -113, 54.21667 ] } }
{ "_id" : ObjectId("5dee03a8379ce4d5c665479b"), "name"

### Filters
Omdat de JSON-file heel veel data bevat, wordt niet alle data geprint. Ergens ook wel logisch, want wat moet je met al die data. Stel dat we alleen de data van meteoriet Kaidun willen opvragen. Daarvoor gebruiken we de query `{"name": "Kaidun"}`.

In [6]:
%%bash
mongo landingsdb --quiet

db.landings.find({"name": "Kaidun"})

{ "_id" : ObjectId("5dee03a8379ce4d5c665494c"), "name" : "Kaidun", "id" : "12228", "nametype" : "Valid", "recclass" : "CR2", "mass" : "2000", "fall" : "Fell", "year" : "1980-01-01T00:00:00.000", "reclat" : "15.000000", "reclong" : "48.300000", "geolocation" : { "type" : "Point", "coordinates" : [ 48.3, 15 ] } }


### Projectie
We zien hier een heleboel gegevens over de meteoriet Kaidun. Veel daarvan (bijvoorbeeld het `id`-veld) hoeven we niet te gebruiken. Stel dat we alleen de naam (`name`), de massa (`mass`) en het jaar (`year`) willen opvragen. Daarvoor gebruiken we projectie. De volgende `find`-opdracht toont alleen de nodige gegevens.

In [7]:
%%bash
mongo landingsdb --quiet

db.landings.find({"name": "Kaidun"}, {"_id":0, "name":1, "mass": 1, "year": 1})

{ "name" : "Kaidun", "mass" : "2000", "year" : "1980-01-01T00:00:00.000" }


### Tabellen
We kunnen dus data opvragen uit onze JSON-file. Maar deze data is nog steeds in JSON-formaat. Stel dat we deze data niet als JSON willen weergeven, maar als een duidelijke tabel. We moeten hiervoor de volgende python modules importeren:

- `os` voor interactie met het OS
- `re` voor het gebruiken van regular expressions
- `numpy` en `pandas` voor het maken van tabellen
- `pymongo` voor het laden van onze data uit MongoDB
- `IPython.core.display` voor het renderen van HTML-tabellen

We importeren deze modules met de volgende regels:

In [8]:
import os
import re
import numpy as np
import pandas as pd
import pymongo as db
import IPython.core.display as view

ModuleNotFoundError: No module named 'numpy'

Maar nu blijkt er dat de modules numpy, pymongo en pandas niet geinstalleerd zijn. Daarom moeten we eerst deze modules installeren. Daarvoor gebruiken we `pip install ...`.

In [9]:
!pip install numpy
!pip install pandas
!pip install pymongo

     |████████████████████████████████| 20.0MB 3.9MB/s eta 0:00:01     |███████████████████▍            | 12.1MB 3.9MB/s eta 0:00:03
     |████████████████████████████████| 10.4MB 6.5MB/s eta 0:00:01    |▉                               | 256kB 6.5MB/s eta 0:00:02
     |████████████████████████████████| 512kB 30.9MB/s eta 0:00:01     |████████████████████████████▎   | 450kB 30.9MB/s eta 0:00:01


Nu we `numpy`, `pymongo` en `pandas` hebben geïnstalleerd, proberen we de packages opnieuw te importeren.

In [10]:
import os
import re
import numpy as np
import pandas as pd
from pymongo import *
import IPython.core.display as view

En het werkt! Nu kunnen we verder met MongoDB en het laden van onze data in tabellen. We testen de aanwezigheid van `MongoDB` met de volgende regel:

In [13]:
# print('Mongo version', pymongo.__version__) # <- deze werkt niet omdat we 'from pymongo import *'

NameError: name 'pymongo' is not defined

Onze database heet `landingsdb`. Deze slaan we voor het gemak op in een variabele. Mochten we de naam willen/moeten veranderen, dan hoeven we alleen deze regels aan te passen.

In [16]:
dbname = "landingsdb"
print("Database name: " + dbname)

Database name: landingsdb


Nu we pymongo hebben geïnstalleerd, kunnen we onze database openen. Met de volgende regels kunnen we onze database openen en kunnen we `find` opdrachten uitvoeren.

In [19]:
# Maak een client: we laden hiermee nog niet onze data.
client = MongoClient('localhost', 27017)
db = client[dbname]
collection = db.landings

# Nu laden we de data in de 'collection'
collection.drop()
os.system('mongoimport -d ' + dbname + ' -c landings meteorite_landings_on_earth.json')

0

In [20]:
cursor = collection.find({})
for obj in cursor:
    print(obj['name'])
    print(obj)

Acapulco
{'_id': ObjectId('5dee044a379ce4d5c6654bac'), 'name': 'Acapulco', 'id': '10', 'nametype': 'Valid', 'recclass': 'Acapulcoite', 'mass': '1914', 'fall': 'Fell', 'year': '1976-01-01T00:00:00.000', 'reclat': '16.883330', 'reclong': '-99.900000', 'geolocation': {'type': 'Point', 'coordinates': [-99.9, 16.88333]}}
Achiras
{'_id': ObjectId('5dee044a379ce4d5c6654bad'), 'name': 'Achiras', 'id': '370', 'nametype': 'Valid', 'recclass': 'L6', 'mass': '780', 'fall': 'Fell', 'year': '1902-01-01T00:00:00.000', 'reclat': '-33.166670', 'reclong': '-64.950000', 'geolocation': {'type': 'Point', 'coordinates': [-64.95, -33.16667]}}
Adhi Kot
{'_id': ObjectId('5dee044a379ce4d5c6654bae'), 'name': 'Adhi Kot', 'id': '379', 'nametype': 'Valid', 'recclass': 'EH4', 'mass': '4239', 'fall': 'Fell', 'year': '1919-01-01T00:00:00.000', 'reclat': '32.100000', 'reclong': '71.800000', 'geolocation': {'type': 'Point', 'coordinates': [71.8, 32.1]}}
Adzhi-Bogdo (stone)
{'_id': ObjectId('5dee044a379ce4d5c6654baf'), '

In [21]:
cursor = collection.find({"name": "Kaidun"})
for obj in cursor:
    print(obj['name'])
    print(obj)

Kaidun
{'_id': ObjectId('5dee044a379ce4d5c6654d60'), 'name': 'Kaidun', 'id': '12228', 'nametype': 'Valid', 'recclass': 'CR2', 'mass': '2000', 'fall': 'Fell', 'year': '1980-01-01T00:00:00.000', 'reclat': '15.000000', 'reclong': '48.300000', 'geolocation': {'type': 'Point', 'coordinates': [48.3, 15]}}


In [22]:
cursor = collection.find({"name": "Kaidun"}, {"_id": 0, "name": 1, "mass": 1, "recclass": 1, "year": 1})
for obj in cursor:
    print(obj['name'])
    print(obj)

Kaidun
{'name': 'Kaidun', 'recclass': 'CR2', 'mass': '2000', 'year': '1980-01-01T00:00:00.000'}


In [23]:
expr = re.compile("kaidun|ställdalen", re.IGNORECASE)
cursor = collection.find({"name": expr}, {"_id": 0, "name": 1, "mass": 1, "recclass": 1, "year": 1})
for obj in cursor:
    print(obj['name'])
    print(obj)

Kaidun
{'name': 'Kaidun', 'recclass': 'CR2', 'mass': '2000', 'year': '1980-01-01T00:00:00.000'}
Ställdalen
{'name': 'Ställdalen', 'recclass': 'H5', 'mass': '34000', 'year': '1876-01-01T00:00:00.000'}


In [47]:
expr = re.compile(".", re.IGNORECASE)
cursor = collection.find({"name": expr}, {"_id": 0, "name": 1, "mass": 1, "year": 1, "reclat": 1, "reclong": 1})
length = 0
for obj in cursor:
    print(obj)
    length += 1
print( "Found " + str(length) + " items" )

{'name': 'Acapulco', 'mass': '1914', 'year': '1976-01-01T00:00:00.000', 'reclat': '16.883330', 'reclong': '-99.900000'}
{'name': 'Achiras', 'mass': '780', 'year': '1902-01-01T00:00:00.000', 'reclat': '-33.166670', 'reclong': '-64.950000'}
{'name': 'Adhi Kot', 'mass': '4239', 'year': '1919-01-01T00:00:00.000', 'reclat': '32.100000', 'reclong': '71.800000'}
{'name': 'Adzhi-Bogdo (stone)', 'mass': '910', 'year': '1949-01-01T00:00:00.000', 'reclat': '44.833330', 'reclong': '95.166670'}
{'name': 'Agen', 'mass': '30000', 'year': '1814-01-01T00:00:00.000', 'reclat': '44.216670', 'reclong': '0.616670'}
{'name': 'Aarhus', 'mass': '720', 'year': '1951-01-01T00:00:00.000', 'reclat': '56.183330', 'reclong': '10.233330'}
{'name': 'Aguada', 'mass': '1620', 'year': '1930-01-01T00:00:00.000', 'reclat': '-31.600000', 'reclong': '-65.233330'}
{'name': 'Aioun el Atrouss', 'mass': '1000', 'year': '1974-01-01T00:00:00.000', 'reclat': '16.398060', 'reclong': '-9.570280'}
{'name': 'Aachen', 'mass': '21', 'ye